In [ ]:
# ==============================================================================
# Step 1: Install and Import Necessary Libraries
# ==============================================================================
# Added 'sentence-transformers' for the new, more powerful model
!pip install datasets scikit-learn numpy pandas sentence-transformers

import time
import numpy as np
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import shuffle

print(f"Setup complete.")

Setup complete.


In [ ]:
# ==============================================================================
# Step 2: Data Loading and Feature Extraction
# ==============================================================================

def load_and_prepare_data(model_name='all-MiniLM-L6-v2'):
    """Loads datasets and converts all sentences to embeddings."""

    # --- Load Datasets ---
    print("Loading English (PAWS) dataset...")
    en_dataset = load_dataset("paws", "labeled_final", split='train[:100000]') # Using a smaller subset for speed
    en_df = en_dataset.to_pandas()
    en_train = en_df.sample(frac=0.8, random_state=42)
    en_test = en_df.drop(en_train.index)
    print(f"English data loaded. Train: {len(en_train)}, Test: {len(en_test)}")

    print("Loading German (PAWS-X) dataset...")
    de_dataset = load_dataset("paws-x", "de", split='train[:100000]')
    de_df = de_dataset.to_pandas()
    de_train = de_df.sample(frac=0.8, random_state=42)
    de_test = de_df.drop(de_train.index)
    print(f"German data loaded. Train: {len(de_train)}, Test: {len(de_test)}")

    # --- Load Sentence Transformer Model ---
    print(f"\nLoading sentence-transformer model: {model_name}...")
    embedder = SentenceTransformer(model_name)
    print("Model loaded.")

    # --- Create Embeddings ---
    print("Converting all sentences to numerical embeddings... (This will take a few minutes)")

    # Collect all unique sentences to encode them only once
    all_sentences = pd.concat([
        en_train['sentence1'], en_train['sentence2'],
        en_test['sentence1'], en_test['sentence2'],
        de_train['sentence1'], de_train['sentence2'],
        de_test['sentence1'], de_test['sentence2']
    ]).unique()

    # Create the embeddings
    embeddings = embedder.encode(all_sentences, show_progress_bar=True)
    embedding_map = {sent: emb for sent, emb in zip(all_sentences, embeddings)}
    print("All sentences have been converted to embeddings.")

    return en_train, en_test, de_train, de_test, embedding_map

# Execute the function
en_train, en_test, de_train, de_test, embedding_map = load_and_prepare_data()

Loading English (PAWS) dataset...
English data loaded. Train: 39521, Test: 9880
Loading German (PAWS-X) dataset...
German data loaded. Train: 39521, Test: 9880

Loading sentence-transformer model: all-MiniLM-L6-v2...
Model loaded.
Converting all sentences to numerical embeddings... (This will take a few minutes)


Batches:   0%|          | 0/2397 [00:00<?, ?it/s]

All sentences have been converted to embeddings.


In [ ]:
# ==============================================================================
# Step 3: Main Execution Block (CORRECTED)
# ==============================================================================
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

def create_feature_vectors(df, embedding_map):
    """Creates combined feature vectors from pre-computed embeddings."""
    s1_embs = np.array([embedding_map[s] for s in df['sentence1']])
    s2_embs = np.array([embedding_map[s] for s in df['sentence2']])
    return np.abs(s1_embs - s2_embs).astype(np.float64)

if __name__ == '__main__':
    # --- Configuration ---
    NUM_ROUNDS = 10
    LOCAL_EPOCHS = 20

    # --- Prepare Datasets with Embeddings ---
    print("\nPreparing feature vectors from embeddings...")
    X_train_en = create_feature_vectors(en_train, embedding_map)
    y_train_en = en_train['label'].values
    X_test_en = create_feature_vectors(en_test, embedding_map)
    y_test_en = en_test['label'].values

    X_train_de = create_feature_vectors(de_train, embedding_map)
    y_train_de = de_train['label'].values
    X_test_de = create_feature_vectors(de_test, embedding_map)
    y_test_de = de_test['label'].values
    print("Feature vectors are ready.")

    # --- Global Model Initialization ---
    print("Initializing the global model (SGDClassifier)...")
    # FIX: Use an adaptive learning rate to allow the model to converge.
    global_model = SGDClassifier(
        loss='log_loss',
        warm_start=True,
        random_state=42,
        learning_rate='optimal' # This will automatically adjust the learning rate
    )

    n_features = X_train_en.shape[1]
    global_model.partial_fit(np.zeros((2, n_features)), np.array([0, 1]), classes=np.array([0, 1]))

    print("Global model initialized.")

    # --- Federated Learning Loop ---
    client_data = {'English': (X_train_en, y_train_en), 'German': (X_train_de, y_train_de)}
    test_data = {'English': (X_test_en, y_test_en), 'German': (X_test_de, y_test_de)}

    for round_num in range(1, NUM_ROUNDS + 1):
        round_start_time = time.time()
        print(f"\n{'='*20} ROUND {round_num}/{NUM_ROUNDS} {'='*20}")

        client_order = list(client_data.keys())
        np.random.shuffle(client_order)

        for client_id in client_order:
            print(f"[Client {client_id}] Starting training...")
            X_train, y_train = client_data[client_id]

            for epoch in range(LOCAL_EPOCHS):
                X_shuffled, y_shuffled = shuffle(X_train, y_train, random_state=epoch)
                global_model.partial_fit(X_shuffled, y_shuffled)

            print(f"[Client {client_id}] Training finished.")

        round_duration = time.time() - round_start_time
        print(f"\n[Server] Round finished in {round_duration:.2f} seconds.")

        # --- Round-wise Evaluation ---
        print(f"\n--- Evaluating Global Model at Round {round_num} ---")
        for client_id, (X_test, y_test) in test_data.items():
            predictions = global_model.predict(X_test)
            accuracy = accuracy_score(y_test, predictions)
            f1 = f1_score(y_test, predictions)
            print(f"  Results for {client_id}: Accuracy={accuracy:.4f}, F1-Score={f1:.4f}")

    print(f"\n{'='*20} FEDERATED TRAINING COMPLETE {'='*20}")


Preparing feature vectors from embeddings...
Feature vectors are ready.
Initializing the global model (SGDClassifier)...
Global model initialized.

==================== ROUND 1/10 ====================
[Client English] Starting training...
[Client English] Training finished.
[Client German] Starting training...
[Client German] Training finished.

[Server] Round finished in 5.05 seconds.

--- Evaluating Global Model at Round 1 ---
  Results for English: Accuracy=0.6174, F1-Score=0.5489
  Results for German: Accuracy=0.6172, F1-Score=0.4258

==================== ROUND 2/10 ====================
[Client German] Starting training...
[Client German] Training finished.
[Client English] Starting training...
[Client English] Training finished.

[Server] Round finished in 4.75 seconds.

--- Evaluating Global Model at Round 2 ---
  Results for English: Accuracy=0.6186, F1-Score=0.4563
  Results for German: Accuracy=0.6161, F1-Score=0.3717

==================== ROUND 3/10 ====================
[Cli